In [2]:
require 'openstudio'
def osload(path)
translator = OpenStudio::OSVersion::VersionTranslator.new
  ospath = OpenStudio::Path.new(path)
  model = translator.loadModel(ospath)
  if model.empty?
    raise "Path '#{path}' is not a valid path to an OpenStudio Model"
  else
    model = model.get
  end
  return model
end

:osload

In [18]:
m0 = osload('C:\Users\HangLi\Altanova\Team - Documents\PROJECTS\Beach Green Dunes III\PSEG-LI\Model 12.3\Beach Green Dunes III ECM1.1.osm')

#<OpenStudio::Model::Model:0x0000000e078290 @__swigtype__="_p_openstudio__model__Model">

In [20]:
m0.getFanOnOffs.each do |fan|
  puts fan.name
  fan.setFanTotalEfficiency(0.88)
end
nil

Fan On Off 9
Fan On Off 65
Fan On Off 63
Fan On Off 36
Fan On Off 37
Fan On Off 19
Fan On Off 1
Fan On Off 54
Fan On Off 2
Fan On Off 32
Fan On Off 45
Fan On Off 7
Fan On Off 34
Fan On Off 3
Fan On Off 42
Fan On Off 40
Fan On Off 4
Fan On Off 20
Fan On Off 8
Fan On Off 55
Fan On Off 6
Fan On Off 58
Fan On Off 5
Fan On Off 57
Fan On Off 10
Fan On Off 11
Fan On Off 12
Fan On Off 14
Fan On Off 31
Fan On Off 28
Fan On Off 13
Fan On Off 50
Fan On Off 41
Fan On Off 15
Fan On Off 16
Fan On Off 17
Fan On Off 61
Fan On Off 52
Fan On Off 18
Fan On Off 49
Fan On Off 21
Fan On Off 64
Fan On Off 22
Fan On Off 23
Fan On Off 48
Fan On Off 24
Fan On Off 51
Fan On Off 25
Fan On Off 26
Fan On Off 27
Fan On Off 29
Fan On Off 30
Fan On Off 33
Fan On Off 35
Fan On Off 38
Fan On Off 46
Fan On Off 39
Fan On Off 43
Fan On Off 44
Fan On Off 47
Fan On Off 53
Fan On Off 56
Fan On Off 59
Fan On Off 60
Fan On Off 62
Fan On Off 66
Fan On Off 67


In [16]:
m0.getCoilHeatingElectrics.each do |coil|
  puts coil.name
  coil.setAvailabilitySchedule(m0.getScheduleRulesetByName('Always Off Schedule').get)
end
nil

Coil Heating Electric 51
Coil Heating Electric 1
Coil Heating Electric 48
Coil Heating Electric 8
Coil Heating Electric 37
Coil Heating Electric 2
Coil Heating Electric 3
Coil Heating Electric 65
Coil Heating Electric 7
Coil Heating Electric 5
Coil Heating Electric 31
Coil Heating Electric 16
Coil Heating Electric 4
Coil Heating Electric 23
Coil Heating Electric 20
Coil Heating Electric 6
Coil Heating Electric 30
Coil Heating Electric 9
Coil Heating Electric 10
Coil Heating Electric 33
Coil Heating Electric 11
Coil Heating Electric 12
Coil Heating Electric 21
Coil Heating Electric 13
Coil Heating Electric 61
Coil Heating Electric 43
Coil Heating Electric 38
Coil Heating Electric 14
Coil Heating Electric 15
Coil Heating Electric 60
Coil Heating Electric 52
Coil Heating Electric 17
Coil Heating Electric 18
Coil Heating Electric 44
Coil Heating Electric 19
Coil Heating Electric 24
Coil Heating Electric 22
Coil Heating Electric 53
Coil Heating Electric 25
Coil Heating Electric 26
Coil Heat

In [23]:
m0.save('C:\Users\HangLi\Altanova\Team - Documents\PROJECTS\Beach Green Dunes III\PSEG-LI\Model 12.3\Beach Green Dunes III ECM1.1.osm',true)

true

In [ ]:
m0 = osload('C:\Users\HangLi\Altanova\Team - Documents\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III baseline.osm')

In [ ]:
# add exhaust fan for baseline (not working because of the nodes)

# m0.getThermalZones.each do |tz|
#   if tz.name.to_s.include?"Apt" or tz.name.to_s.include?"apt"
#     puts tz.name
#     fan = m0.getFanZoneExhausts[0].clone(m0)
#     tz.addEquipment(fan)
#   end
# end
# nil
# m0.save('Beach Green Dunes III baseline.osm',true)

In [ ]:
m = osload('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ECM1.1.osm')

In [ ]:
m.getSetpointManagers.each do |set|
  puts set.name
end
nil

In [ ]:
puts m.getSetpointManagers[2]

In [ ]:
# REMOVE EMS
class OpenStudio::Model::ModelObject
  # from Julien Marrec
  # https://unmethours.com/question/17616/get-thermal-zone-supply-terminal/
  
  # Extend ModelObject class to add a to_actual_object method
  # Casts a ModelObject into what it actually is (OS:Node for example...)
  def to_actual_object
    obj_type = self.iddObjectType.valueName
    obj_type_name = obj_type.gsub('OS_','').gsub('_','')
    method_name = "to_#{obj_type_name}"
    if self.respond_to?(method_name)
      actual_thing = self.method(method_name).call
      if !actual_thing.empty?
          return actual_thing.get
      end
    end
    return false
  end
  
end

In [ ]:
m0.modelObjects.each do |obj|    
  if obj.to_actual_object.class.to_s.include?('EnergyManagementSystem')
    puts obj.to_actual_object.class.to_s
    count += 1
  end
end
nil

# ECM 1.1 GSHP


### 2. apply DivisionOnly (a bug in OS 2.9.0)


In [2]:
m0 = osload('C:\Users\HangLi\Altanova\Team - Documents\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ECM1.1.osm')

#<OpenStudio::Model::Model:0x0000000720fa60 @__swigtype__="_p_openstudio__model__Model">

In [3]:
# reset Heating COP to 4.2
m0.getCoilHeatingWaterToAirHeatPumpVariableSpeedEquationFitSpeedDatas.each do |d|
  puts d.name
  d.setReferenceUnitGrossRatedHeatingCOP(4.2)
end
nil

In [ ]:
### TO DO ###
#1. SET DOAS cooling COP to 5.2
#2. SET DOAS fan total efficiency to 0.8

In [ ]:
# remove dcv
m0.getAirLoopHVACs.each do |air_loop|
  air_loop.supplyComponents.each do |supply_component|
    hVACComponent = supply_component.to_AirLoopHVACOutdoorAirSystem
    if hVACComponent.is_initialized
      hVACComponent = hVACComponent.get
      controller_oa = hVACComponent.getControllerOutdoorAir
      controller_mv = controller_oa.controllerMechanicalVentilation    
      controller_mv.setDemandControlledVentilation(false)
    end
  end
end
nil

In [ ]:
# remove ERV
m0.getHeatExchangerAirToAirSensibleAndLatents.each do |erv|
  puts erv.name
  erv.remove
end
nil

In [22]:
# change hvac priority:  1. outdoor air system (vav) 2. heat pump
m0.getThermalZones.each do |z|
  z.equipment.each do |e|
    if e.to_AirTerminalSingleDuctConstantVolumeNoReheat.is_initialized
      puts "Switching VAVNoReheat to first priority for #{z.name.to_s}"
      z.setHeatingPriority(e, 1)
      z.setCoolingPriority(e, 1)
    end
  end
end
nil

Switching VAVNoReheat to first priority for Thermal Zone: F6 Apt 1
Switching VAVNoReheat to first priority for Thermal Zone: F5 Apt 2
Switching VAVNoReheat to first priority for Thermal Zone: F6 Apt 2
Switching VAVNoReheat to first priority for Thermal Zone: F3 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F8 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F8 Apt 1
Switching VAVNoReheat to first priority for Thermal Zone: F5 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F7 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F2 Apt 2
Switching VAVNoReheat to first priority for Thermal Zone: GF Apt 1
Switching VAVNoReheat to first priority for Thermal Zone: F6 Apt 3
Switching VAVNoReheat to first priority for Thermal Zone: F8 Apt 3
Switching VAVNoReheat to first priority for Thermal Zone: F2 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F2 Apt 3
Switching VAVNoReheat to first priority for Thermal Zone: F7 A

In [ ]:
# replace unitary system with DX cooling and DX heating
m0.getAirLoopHVACs.each do |airloop|
  puts airloop.name
  unitary_systems = airloop.supplyComponents("OS:AirLoopHVAC:UnitarySystem".to_IddObjectType)
  u1= unitary_systems[0].to_AirLoopHVACUnitarySystem.get
  u2 = unitary_systems[1].to_AirLoopHVACUnitarySystem.get
  
  node1 = u1.inletNode.get
  u1.remove
  coil_dxcooling = OpenStudio::Model::CoilCoolingDXTwoSpeed.new(m0)
  coil_dxcooling.addToNode(node1)
  
  node2 = u2.inletNode.get
  u2.remove
  coil_dxheating = OpenStudio::Model::CoilHeatingDXSingleSpeed.new(m0)
  coil_dxheating.addToNode(node2)
  
end
nil

In [ ]:
# change schedules: AEDG K-12 Ventilation Schedule
vent_sche=m0.getScheduleRulesetByName('AEDG K-12 Ventilation Schedule').get
vent_daysche = vent_sche.defaultDaySchedule
vent_daysche.clearValues()
vent_daysche.addValue(OpenStudio::Time.new(0,24,0,0),1)

# delete weekend and summer schedules
vent_sche.scheduleRules.each do |sr|
  puts sr.name
  sr.remove
end
nil

In [5]:
# heat pump all OA autosized
# blowthrough --> drawthrough

m0.getZoneHVACWaterToAirHeatPumps.each do |hp|
  hp.autosizeSupplyAirFlowRateWhenNoCoolingorHeatingisNeeded()
  hp.autosizeOutdoorAirFlowRateDuringCoolingOperation()
  hp.autosizeOutdoorAirFlowRateDuringHeatingOperation()
  hp.autosizeOutdoorAirFlowRateWhenNoCoolingorHeatingisNeeded()
  hp.setFanPlacement("DrawThrough")
end
nil


In [6]:
m0.save('Beach Green Dunes III ECM1.1 - updated.osm',true)

true

# ECM1.2 DOAS


In [95]:
m0 = osload('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ECM1.1.osm')

#<OpenStudio::Model::Model:0x00000005489590 @__swigtype__="_p_openstudio__model__Model">

In [82]:
# # create controller outdoor air
# controller_OA = OpenStudio::Model::ControllerOutdoorAir.new(model)
# controller_OA.autosizeMinimumOutdoorAirFlowRate()
# controller_OA.autosizeMaximumOutdoorAirFlowRate()
# controller_OA.setEconomizerControlType("DifferentialEnthalpy")
# # controller_OA.setMaximumFractionofOutdoorAirSchedule(options["ventilation_schedule"])
# controller_OA.setHeatRecoveryBypassControlType("BypassWhenOAFlowGreaterThanMinimum")
# # create outdoor air system
# system_OA = OpenStudio::Model::AirLoopHVACOutdoorAirSystem.new(model, controller_OA)
# # air_loop_comps << system_OA
# # create ERV
# heat_exchanger = OpenStudio::Model::HeatExchangerAirToAirSensibleAndLatent.new(model)
# heat_exchanger.setAvailabilitySchedule(model.alwaysOnDiscreteSchedule())
# sensible_eff = 0.75
# latent_eff = 0.69
# heat_exchanger.setSensibleEffectivenessat100CoolingAirFlow(sensible_eff)
# heat_exchanger.setSensibleEffectivenessat100HeatingAirFlow(sensible_eff)
# heat_exchanger.setSensibleEffectivenessat75CoolingAirFlow(sensible_eff)
# heat_exchanger.setSensibleEffectivenessat75HeatingAirFlow(sensible_eff)
# heat_exchanger.setLatentEffectivenessat100CoolingAirFlow(latent_eff)
# heat_exchanger.setLatentEffectivenessat100HeatingAirFlow(latent_eff)
# heat_exchanger.setLatentEffectivenessat75CoolingAirFlow(latent_eff)
# heat_exchanger.setLatentEffectivenessat75HeatingAirFlow(latent_eff)
# heat_exchanger.setFrostControlType("ExhaustOnly")
# heat_exchanger.setThresholdTemperature(-12.2)
# heat_exchanger.setInitialDefrostTimeFraction(0.1670)
# heat_exchanger.setRateofDefrostTimeFractionIncrease(0.0240)
# heat_exchanger.setEconomizerLockout(false)
# # create setpoint manager for airloop
# setpoint_manager = OpenStudio::Model::SetpointManagerSingleZoneReheat.new(model)
# setpoint_manager.setMinimumSupplyAirTemperature(10)
# setpoint_manager.setMaximumSupplyAirTemperature(50)
# # setpoint_manager.setControlZone(zone)

In [83]:
# create DOAS loop
airloop = OpenStudio::Model::addSystemType4(m0).to_AirLoopHVAC.get
sizingsystem = airloop.sizingSystem
sizingsystem.setTypeofLoadtoSizeOn("VentilationRequirement") 
sizingsystem.setAllOutdoorAirinCooling(true) 
sizingsystem.setAllOutdoorAirinHeating(true) 

true

In [77]:
# # create fan
# fan = OpenStudio::Model::FanVariableVolume.new(model, model.alwaysOnDiscreteSchedule())
# fan.setFanEfficiency(0.6)
# fan.setPressureRise(fan_pressure_rise) #Pa
# fan.autosizeMaximumFlowRate()
# fan.setFanPowerMinimumFlowFraction(0.6)
# fan.setMotorEfficiency(0.85)
# fan.setMotorInAirstreamFraction(1.0)

In [84]:
m0.getThermalZones.each do |tz|
  if tz.name.to_s.include?"Apt" or tz.name.to_s.include?"apt"
    puts tz.name
    # create terminal
    air_terminal = OpenStudio::Model::AirTerminalSingleDuctVAVNoReheat.new(m0, m0.alwaysOnDiscreteSchedule())
    air_terminal.setControlForOutdoorAir(true)
    air_terminal.setConstantMinimumAirFlowFraction(0)
    # Add Apt. zones to DOAS loop
    airloop.addBranchForZone(tz, air_terminal.to_StraightComponent)
    
    # turn off hp ventilation
    hp = tz.equipment[0].to_ZoneHVACWaterToAirHeatPump.get
    hp.setSupplyAirFlowRateWhenNoCoolingorHeatingisNeeded(0)
    hp.setOutdoorAirFlowRateDuringCoolingOperation(0)
    hp.setOutdoorAirFlowRateDuringHeatingOperation(0)
    hp.setOutdoorAirFlowRateWhenNoCoolingorHeatingisNeeded(0)
  end
end
nil

Thermal Zone: F6 Apt 1
Thermal Zone: F5 Apt 2
Thermal Zone: F6 Apt 2
Thermal Zone: F3 Apt 4
Thermal Zone: F8 Apt 4
Thermal Zone: F8 Apt 1
Thermal Zone: F5 Apt 4
Thermal Zone: F7 Apt 4
Thermal Zone: F2 Apt 2
Thermal Zone: GF Apt 1
Thermal Zone: F6 Apt 3
Thermal Zone: F8 Apt 3
Thermal Zone: F2 Apt 4
Thermal Zone: F2 Apt 3
Thermal Zone: F7 Apt 2
Thermal Zone: F5 Apt 1
Thermal Zone: F3 Apt 1
Thermal Zone: F3 Apt 2
Thermal Zone: F3 Apt 3
Thermal Zone: F4 Apt 1
Thermal Zone: F5 Apt 3
Thermal Zone: F4 Apt 2
Thermal Zone: F4 Apt 3
Thermal Zone: F5 Apt 5
Thermal Zone: F7 Apt 5
Thermal Zone: F4 Apt 4
Thermal Zone: F6 Apt 5
Thermal Zone: F6 Apt 4
Thermal Zone: F7 Apt 3
Thermal Zone: F7 Apt 1
Thermal Zone: F8 Apt 2
Thermal Zone: F8 Apt 5
Thermal Zone: F2 Apt


In [85]:
# create DOAS loop for corridor
airloop_2 = OpenStudio::Model::addSystemType4(m0).to_AirLoopHVAC.get
sizingsystem_2 = airloop_2.sizingSystem
sizingsystem_2.setTypeofLoadtoSizeOn("VentilationRequirement") 
sizingsystem_2.setAllOutdoorAirinCooling(true) 
sizingsystem_2.setAllOutdoorAirinHeating(true) 

true

In [86]:
m0.getThermalZones.each do |tz|
  if tz.name.to_s.include?"corr" or tz.name.to_s.include?"Cor"
    puts tz.name
    # create terminal
    air_terminal = OpenStudio::Model::AirTerminalSingleDuctVAVNoReheat.new(m0, m0.alwaysOnDiscreteSchedule())
    air_terminal.setControlForOutdoorAir(true)
    air_terminal.setConstantMinimumAirFlowFraction(0)
    # Add Apt. zones to DOAS loop
    airloop_2.addBranchForZone(tz, air_terminal.to_StraightComponent)
    
    # turn off hp ventilation
    hp = tz.equipment[0].to_ZoneHVACWaterToAirHeatPump.get
    hp.setSupplyAirFlowRateWhenNoCoolingorHeatingisNeeded(0)
    hp.setOutdoorAirFlowRateDuringCoolingOperation(0)
    hp.setOutdoorAirFlowRateDuringHeatingOperation(0)
    hp.setOutdoorAirFlowRateWhenNoCoolingorHeatingisNeeded(0)
  end
end
nil

Thermal Zone: F4 corridor 2
Thermal Zone: F5 corridor
Thermal Zone: F4 corridor 1
Thermal Zone: GF corridor
Thermal Zone: F2 corridor 2
Thermal Zone: F6 corridor
Thermal Zone: F3 corridor 2
Thermal Zone: F3 corridor 1
Thermal Zone: F7 corridor
Thermal Zone: F8 corridor
Thermal Zone: F2 corridor
Thermal Zone: GF corridor 3


In [87]:
# change hvac priority:   1. outdoor air system 2. heat pump 
m0.getThermalZones.each do |z|
  z.equipment.each do |e|
    if e.to_AirTerminalSingleDuctVAVNoReheat.is_initialized
      puts "Switching VAVNoReheat to first priority for #{z.name.to_s}"
      z.setHeatingPriority(e, 1)
      z.setCoolingPriority(e, 1)
    end
  end
end
nil

Switching VAVNoReheat to first priority for Thermal Zone: F5 Apt 2
Switching VAVNoReheat to first priority for Thermal Zone: F6 Apt 2
Switching VAVNoReheat to first priority for Thermal Zone: F3 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F5 corridor
Switching VAVNoReheat to first priority for Thermal Zone: F8 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F8 Apt 1
Switching VAVNoReheat to first priority for Thermal Zone: F5 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F7 Apt 4
Switching VAVNoReheat to first priority for Thermal Zone: F2 Apt 2
Switching VAVNoReheat to first priority for Thermal Zone: F4 corridor 1
Switching VAVNoReheat to first priority for Thermal Zone: GF Apt 1
Switching VAVNoReheat to first priority for Thermal Zone: F6 Apt 3
Switching VAVNoReheat to first priority for Thermal Zone: GF corridor
Switching VAVNoReheat to first priority for Thermal Zone: F2 corridor 2
Switching VAVNoReheat to first priority for Th

In [88]:
m0.save('Beach Green Dunes III ECM1.2 DOAS.osm')

true

# ECM1.2 ERV

### #Plate
### #Supply Air Outlet Temprature Control: yes

In [3]:
m0 = osload('Beach Green Dunes III ECM1.1.osm')

#<OpenStudio::Model::Model:0x00000006ef00a0 @__swigtype__="_p_openstudio__model__Model">

In [4]:
m0.getAirLoopHVACs.each do |airloop|
  puts airloop.name
  oa_node = airloop.airLoopHVACOutdoorAirSystem.get.outboardOANode.get
  hx = OpenStudio::Model::HeatExchangerAirToAirSensibleAndLatent.new(m0)
  hx.addToNode(oa_node)
end
nil
  


DOAS


In [5]:
m0.save('Beach Green Dunes III ECM1.2.osm',true)

true

# ECM 1.2 Economizer (not saving)

In [6]:
# m0.getAirLoopHVACs.each do |air_loop|
#   air_loop.supplyComponents.each do |supply_component|
#     hVACComponent = supply_component.to_AirLoopHVACOutdoorAirSystem
#     if hVACComponent.is_initialized
#       hVACComponent = hVACComponent.get
#       controller_oa = hVACComponent.getControllerOutdoorAir
#       starting_econo_control_type = controller_oa.getEconomizerControlType
#       puts starting_econo_control_type
#       controller_oa.setEconomizerControlType('FixedEnthalpy')
#       final_econo_control_type = controller_oa.getEconomizerControlType
#       puts final_econo_control_type
#     end
#   end
# end
# nil

NoEconomizer
FixedEnthalpy


In [7]:
# m0.save('Beach Green Dunes III ECM1.4.osm', true)

true

# ECM 1.3 DCV (enthalpy) 

In [10]:
m0 = osload('Beach Green Dunes III ECM1.3.osm')

#<OpenStudio::Model::Model:0x000000064ec4c8 @__swigtype__="_p_openstudio__model__Model">

In [7]:
# replace constant fan with variable speed

In [11]:
m0.getAirTerminalSingleDuctVAVNoReheats.each do |air_terminal|
  puts air_terminal.name
  air_terminal.setControlForOutdoorAir(true)
end
nil

Air Terminal Single Duct VAV No Reheat 1
VAV No Rht
VAV No Rht 4
VAV No Rht 1
VAV No Rht 25
VAV No Rht 2
VAV No Rht 6
VAV No Rht 32
VAV No Rht 9
VAV No Rht 3
VAV No Rht 14
VAV No Rht 8
VAV No Rht 7
VAV No Rht 19
VAV No Rht 5
VAV No Rht 10
VAV No Rht 11
VAV No Rht 12
VAV No Rht 13
VAV No Rht 15
VAV No Rht 16
VAV No Rht 17
VAV No Rht 18
VAV No Rht 20
VAV No Rht 30
VAV No Rht 21
VAV No Rht 22
VAV No Rht 23
VAV No Rht 24
VAV No Rht 26
VAV No Rht 27
VAV No Rht 28
VAV No Rht 29
VAV No Rht 31


In [12]:
m0.getAirLoopHVACs.each do |air_loop|
  air_loop.supplyComponents.each do |supply_component|
    hVACComponent = supply_component.to_AirLoopHVACOutdoorAirSystem
    if hVACComponent.is_initialized
      hVACComponent = hVACComponent.get
      controller_oa = hVACComponent.getControllerOutdoorAir
      controller_mv = controller_oa.controllerMechanicalVentilation    
      controller_mv.setDemandControlledVentilation(true)
    end
  end
end
nil

In [13]:
m0.save('Beach Green Dunes III ECM1.3.osm', true)

true

# ECM 1.3 Thermal Comfort Ranges

In [14]:
htg_sch=m0.getScheduleRulesetByName('MidriseApartment Apartment HtgSetp').get
clg_sch=m0.getScheduleRulesetByName('MidriseApartment Apartment ClgSetp').get

htg_daysch = htg_sch.defaultDaySchedule
clg_daysch = clg_sch.defaultDaySchedule

htg_daysch.addValue(OpenStudio::Time.new(0,6,0,0),OpenStudio.convert(68, "F", "C").get)
htg_daysch.addValue(OpenStudio::Time.new(0,23,0,0),OpenStudio.convert(70, "F", "C").get)
htg_daysch.addValue(OpenStudio::Time.new(0,24,0,0),OpenStudio.convert(68, "F", "C").get)

clg_daysch.addValue(OpenStudio::Time.new(0,9,0,0),OpenStudio.convert(80, "F", "C").get)
clg_daysch.addValue(OpenStudio::Time.new(0,15,0,0),OpenStudio.convert(82, "F", "C").get)
clg_daysch.addValue(OpenStudio::Time.new(0,24,0,0),OpenStudio.convert(80, "F", "C").get)

true

In [15]:
m0.save('Beach Green Dunes III ECM1.4.osm', true)

true

# ECM 1.4 Add smart control

In [16]:
htg_sch=m0.getScheduleRulesetByName('MidriseApartment Apartment HtgSetp').get
clg_sch=m0.getScheduleRulesetByName('MidriseApartment Apartment ClgSetp').get

htg_daysch = htg_sch.defaultDaySchedule
clg_daysch = clg_sch.defaultDaySchedule

htg_daysch.addValue(OpenStudio::Time.new(0,6,0,0),OpenStudio.convert(68, "F", "C").get)
htg_daysch.addValue(OpenStudio::Time.new(0,8,0,0),OpenStudio.convert(70, "F", "C").get)
htg_daysch.addValue(OpenStudio::Time.new(0,12,0,0),OpenStudio.convert(60, "F", "C").get)
htg_daysch.addValue(OpenStudio::Time.new(0,13,0,0),OpenStudio.convert(65, "F", "C").get)
htg_daysch.addValue(OpenStudio::Time.new(0,23,0,0),OpenStudio.convert(70, "F", "C").get)
htg_daysch.addValue(OpenStudio::Time.new(0,24,0,0),OpenStudio.convert(68, "F", "C").get)


clg_daysch.addValue(OpenStudio::Time.new(0,8,0,0),OpenStudio.convert(80, "F", "C").get)
clg_daysch.addValue(OpenStudio::Time.new(0,12,0,0),OpenStudio.convert(90, "F", "C").get)
clg_daysch.addValue(OpenStudio::Time.new(0,13,0,0),OpenStudio.convert(85, "F", "C").get)
clg_daysch.addValue(OpenStudio::Time.new(0,24,0,0),OpenStudio.convert(80, "F", "C").get)

true

In [17]:
m0.save('Beach Green Dunes III ECM1.5.osm', true)

true

# ECM1.5 Fan with EC motors

In [18]:
m0.getFanVariableVolumes.each do |fan|
  puts fan.name
  
  m = fan.motorEfficiency
  e = fan.fanTotalEfficiency
  
  fan.setMotorEfficiency(0.93)
#   fan.setFanTotalEfficiency(e*0.93/m)
  fan.setFanTotalEfficiency(0.9)  # 0.85 --> 0.9
end
nil
  
  

Var Spd Fan


In [19]:
m0.save('Beach Green Dunes III ECM1.6.osm', true)

true

# ECM 2.1 AGW

In [ ]:
# # BGW
# m0.getSurfaces.each do |s|
#   if s.construction.get.name.to_s.include?"Typical Insulated Basement Mass Wall"
#     puts s.name
#     s.setConstruction(m0.getConstructionByName('Proposed BGW').get)
#   end
#   end
# nil

In [20]:
# ABG
m0.getSurfaces.each do |s|
  if s.construction.get.name.to_s.include?"Typical Insulated Steel Framed Exterior Wall R-15.63"
    puts s.name
    s.setConstruction(m0.getConstructionByName('Proposed Exterior Wall').get)
  end
  end
nil

Surface 841
Surface 242
Surface 32
Surface 615
Surface 31
Surface 432
Surface 352
Surface 249
Surface 730
Surface 205
Surface 230
Surface 822
Surface 330
Surface 194
Surface 697
Surface 200
Surface 198
Surface 860
Surface 425
Surface 113
Surface 153
Surface 107
Surface 156
Surface 589
Surface 146
Surface 98
Surface 180
Surface 178
Surface 821
Surface 152
Surface 179
Surface 91
Surface 181
Surface 855
Surface 193
Surface 474
Surface 25
Surface 698
Surface 199
Surface 95
Surface 827
Surface 211
Surface 212
Surface 41
Surface 218
Surface 857
Surface 636
Surface 342
Surface 233
Surface 165
Surface 315
Surface 236
Surface 237
Surface 628
Surface 241
Surface 243
Surface 244
Surface 423
Surface 254
Surface 94
Surface 257
Surface 258
Surface 578
Surface 532
Surface 264
Surface 265
Surface 839
Surface 331
Surface 270
Surface 350
Surface 271
Surface 288
Surface 394
Surface 289
Surface 484
Surface 290
Surface 313
Surface 306
Surface 531
Surface 305
Surface 590
Surface 307
Surface 309
Surface 850


In [21]:
m0.save('Beach Green Dunes III ECM2.1.osm', true)

true

# ECM 2.2 High performance roof

In [22]:
# Roof
m0.getSurfaces.each do |s|
  if s.construction.get.name.to_s.include?"Typical IEAD Roof R-31.25"
    puts s.name
    s.setConstruction(m0.getConstructionByName('Proposed Roof').get)
  end
  end
nil

Surface 625
Surface 195
Surface 776
Surface 722
Surface 38
Surface 8
Surface 591
Surface 579
Surface 597
Surface 619
Surface 220
Surface 631
Surface 716
Surface 719
Surface 725
Surface 726
Surface 92
Surface 122


In [23]:
m0.save('Beach Green Dunes III ECM2.2.osm', true)

true

In [24]:
# I changed the roof material

# ECM 2.3 Window

In [25]:
# Windows
m0.getSubSurfaces.each do |ss|
  puts ss.name
  puts ss.construction.get.name
  ss.setConstruction(m0.getConstructionByName('Proposed Window').get)
  puts ss.construction.get.name

end
nil

Sub Surface 137
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 79
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 13
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 70
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 63
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 4
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 78
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 8
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 68
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 10
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 14
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 26
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 110
ASHRAE 189.1-2009 ExtWindow ClimateZone 3
Proposed Window
Sub Surface 11
ASHRAE 189.1-2009 ExtWindow ClimateZ

In [26]:
m0.save('Beach Green Dunes III ECM2.3.osm', true)

true

# ECM 2.4 Electrochromic Glazing

# ECM 3.1 LED

In [27]:
m0.getLightsDefinitions.each do |l|
  lpd_new = l.wattsperSpaceFloorArea.get * 0.6
  puts lpd_new
  l.setWattsperSpaceFloorArea(lpd_new)
end
nil

4.520842375018086
3.875007750015498
7.7500155000310205
3.229173125012916
8.39585012503356
9.687519375038761
5.81251162502325
5.166677000020668


In [28]:
# set for apartment
l = m0.getLightsDefinitionByName('189.1-2009 - MidriseApartment - Apartment - CZ4-8 Additional Lights Definition').get
l.setWattsperSpaceFloorArea(0.4*10.7639)

true

In [29]:
m0.save('Beach Green Dunes III ECM3.1.osm', true)

true

# ECM 3.2 occupancy sensor

space types have reduction:


In [30]:
m0.getLightsDefinitions.each do |l|
  if l.name.to_s.include?"189.1-2009 - MidriseApartment - Apartment - CZ4-8 Additional Lights Definition" 
    puts l.name
    lpd_new = l.wattsperSpaceFloorArea.get * 0.67
    l.setWattsperSpaceFloorArea(lpd_new)
  end
  
  if l.name.to_s.include?"BreakRoom" or l.name.to_s.include?"Corridor" or l.name.to_s.include?"Lobby" or l.name.to_s.include?"Restroom" or l.name.to_s.include?"Stair"
    puts l.name
    lpd_new = l.wattsperSpaceFloorArea.get * 0.28
    l.setWattsperSpaceFloorArea(lpd_new)
  end
end
nil

189.1-2009 - MidriseApartment - Apartment - CZ4-8 Additional Lights Definition
189.1-2009 - Office - Stair - CZ4-8 Lights Definition
189.1-2009 - Office - BreakRoom - CZ4-8 Lights Definition
189.1-2009 - Office - Corridor - CZ4-8 Lights Definition
189.1-2009 - Office - Lobby - CZ4-8 Lights Definition
189.1-2009 - Office - Restroom - CZ4-8 Lights Definition


In [31]:
m0.save('Beach Green Dunes III ECM3.2.osm', true)

true

# ECM 3.3 Daylight Sensor  (removed)
### need to apply to all spaces

In [163]:
# m0 = osload('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ECM3.2.osm')

In [164]:
# new_sensor_objects = {}
# affected_zones = []
# affected_zone_names = []
# area = 0

# m0.getSpaces.each do |space|
#   puts space.name
#   area += space.floorArea
  
#   # eliminate spaces that don't have exterior natural lighting
#   has_ext_nat_light = false
#   space.surfaces.each do |surface|
#     next if surface.outsideBoundaryCondition != 'Outdoors'
#     surface.subSurfaces.each do |sub_surface|
#       next if sub_surface.subSurfaceType == 'Door'
#       next if sub_surface.subSurfaceType == 'OverheadDoor'
#       has_ext_nat_light = true
#     end
#   end

#   # find floors
#   floors = []
#   space.surfaces.each do |surface|
#     next if surface.surfaceType != 'Floor'
#     floors << surface
#   end

#   # this method only works for flat (non-inclined) floors
#   boundingBox = OpenStudio::BoundingBox.new
#   floors.each do |floor|
#     boundingBox.addPoints(floor.vertices)
#   end
#   xmin = boundingBox.minX.get
#   ymin = boundingBox.minY.get
#   zmin = boundingBox.minZ.get
#   xmax = boundingBox.maxX.get
#   ymax = boundingBox.maxY.get

#   # create a new sensor and put at the center of the space
#   sensor = OpenStudio::Model::DaylightingControl.new(m0)
#   sensor.setName("#{space.name} daylighting control")
#   x_pos = (xmin + xmax) / 2
#   y_pos = (ymin + ymax) / 2
#   z_pos = zmin + 0.762 # put it 1 meter above the floor
#   new_point = OpenStudio::Point3d.new(x_pos, y_pos, z_pos)
#   sensor.setPosition(new_point)
#   sensor.setIlluminanceSetpoint(484)
#   sensor.setLightingControlType('Continuous/Off')
#   sensor.setMinimumInputPowerFractionforContinuousDimmingControl(0)
#   sensor.setMinimumLightOutputFractionforContinuousDimmingControl(0)
#   sensor.setSpace(space)
#   puts sensor.setSpace(space)
  
#   temp_zone = space.thermalZone.get
#   affected_zones << temp_zone
#   affected_zone_names << temp_zone.name.to_s
  
#   new_sensor_objects[temp_zone.name.to_s] = sensor

# end
# nil

In [165]:
# affected_zones.each do |zone|
#   zone.setPrimaryDaylightingControl(new_sensor_objects[zone.name.to_s])
#   zone.setFractionofZoneControlledbyPrimaryDaylightingControl(1)
# end
# nil

In [166]:
# m0.save('Beach Green Dunes III ECM3.3.osm', true)

# ECM 4.1 High efficiency equipment (EnergyStar refrigerator, cloth washer, dishwasher)

### Apt: 0.97 W/ft2 --> 0.86W/ft2
### laundry: 27.95 W/ft2 --> 27.16W/ft2

In [32]:
d1 = m0.getElectricEquipmentDefinitionByName("189.1-2009 - MidriseApartment - Apartment - CZ4-8 Electric Equipment Definition").get
d1.setWattsperSpaceFloorArea(0.86 * 10.7639)
d2 = m0.getElectricEquipmentDefinitionByName("Laundry Electric Equipment Definition").get
d2.setWattsperSpaceFloorArea(27.16 * 10.7639)

true

In [33]:
water = m0.getWaterUseEquipmentDefinitionByName("Water Fixture Definition").get
water.setPeakFlowRate(water.peakFlowRate * 11.4 / 12.4) # dish washer (energy star)

true

In [34]:
m0.save('Beach Green Dunes III ECM4.1.osm', true)

true

# ECM 4.2

In [35]:
water = m0.getWaterUseEquipmentDefinitionByName("Water Fixture Definition").get
water.setPeakFlowRate(water.peakFlowRate * 8.6 / 11.4) # (toilet, faucets, showerhead)

true

In [36]:
m0.save('Beach Green Dunes III ECM4.2.osm', true)

true

# ECM 4.3 Low Flow Fixture

### peak flow rate 

In [37]:
water = m0.getWaterUseEquipmentDefinitionByName("Water Fixture Definition").get
water.setPeakFlowRate(water.peakFlowRate * 6.9 / 8.6) # (toilet, faucets, showerhead)


true

In [38]:
m0.save('Beach Green Dunes III ECM4.3.osm', true)

true

# Extra code (for updating all models)

In [46]:
def update_plug_load(name)
  m = osload('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm')
  d1 = m.getElectricEquipmentDefinitionByName("189.1-2009 - MidriseApartment - Apartment - CZ4-8 Electric Equipment Definition").get
  d1.setWattsperSpaceFloorArea(0.97*10.7639)
  d2 = m.getElectricEquipmentDefinitionByName("Laundry Electric Equipment Definition").get
  d2.setWattsperSpaceFloorArea(27.95*10.7639)
  m.save('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm',true)
end

:update_plug_load

In [47]:
update_plug_load('baseline')

true

In [48]:
['ECM1.1','ECM1.2','ECM1.3','ECM1.4','ECM1.5','ECM1.6','ECM1.7'].each do |m|
  update_plug_load(m)
end

["ECM1.1", "ECM1.2", "ECM1.3", "ECM1.4", "ECM1.5", "ECM1.6", "ECM1.7"]

In [49]:
['ECM2.1','ECM2.2','ECM2.3','ECM3.1','ECM3.2','ECM3.3'].each do |m|
  update_plug_load(m)
end

["ECM2.1", "ECM2.2", "ECM2.3", "ECM3.1", "ECM3.2", "ECM3.3"]

In [50]:
def update_flow_fixture(name)
  m = osload('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm')
  water = m.getWaterUseEquipmentDefinitionByName("Water Fixture Definition").get
  water.setPeakFlowRate(water.peakFlowRate * 12.4 / 10.4)
  m.save('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm',true)
end

:update_flow_fixture

In [51]:
['ECM1.1','ECM1.2','ECM1.3','ECM1.4','ECM1.5','ECM1.6','ECM1.7'].each do |m|
  update_flow_fixture(m)
end

["ECM1.1", "ECM1.2", "ECM1.3", "ECM1.4", "ECM1.5", "ECM1.6", "ECM1.7"]

In [52]:
['ECM2.1','ECM2.2','ECM2.3','ECM3.1','ECM3.2','ECM3.3'].each do |m|
  update_flow_fixture(m)
end

["ECM2.1", "ECM2.2", "ECM2.3", "ECM3.1", "ECM3.2", "ECM3.3"]

In [53]:
update_flow_fixture('baseline')

true

In [57]:
# set for apartment
model_name = 'D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ECM4.3.osm'
m0 = osload(model_name)
l = m0.getLightsDefinitionByName('189.1-2009 - MidriseApartment - Apartment - CZ4-8 Additional Lights Definition').get
l.setWattsperSpaceFloorArea(0.4*10.7639)
m0.save(model_name, true)

true

In [58]:
p = m0.getElectricEquipmentDefinitions[0]

#<OpenStudio::Model::ElectricEquipmentDefinition:0x0000000db7c728 @__swigtype__="_p_openstudio__model__ElectricEquipmentDefinition">

In [66]:
p.wattsperSpaceFloorArea.get

292.347524

In [70]:
def update_plug_load(name)
  m = osload('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm')
  plug_load = m.getElectricEquipmentDefinitionByName("Laundry Electric Equipment Definition").get
  plug_load.setWattsperSpaceFloorArea(13.16 * 10.7639)
  m.save('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm',true)
end

:update_plug_load

In [71]:
10.7639

10.7639

In [72]:
['baseline','ECM1.1','ECM1.2','ECM1.3','ECM1.4','ECM1.5','ECM2.1','ECM2.2','ECM2.3','ECM2.4','ECM3.1','ECM3.2','ECM3.3'].each do |m|
  update_plug_load(m)
end

["baseline", "ECM1.1", "ECM1.2", "ECM1.3", "ECM1.4", "ECM1.5", "ECM2.1", "ECM2.2", "ECM2.3", "ECM2.4", "ECM3.1", "ECM3.2", "ECM3.3"]

In [73]:
def update_plug_load_1(name)
  m = osload('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm')
  plug_load = m.getElectricEquipmentDefinitionByName("Laundry Electric Equipment Definition").get
  plug_load.setWattsperSpaceFloorArea(12.59 * 10.7639)
  m.save('D:\Box Sync\PROJECTS\Beach Green Dunes III\PSEG-LI\Model\Beach Green Dunes III ' + name + '.osm',true)
end

:update_plug_load_1

In [74]:
['ECM4.1','ECM4.2','ECM4.3'].each do |m|
  update_plug_load_1(m)
end

["ECM4.1", "ECM4.2", "ECM4.3"]